In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib

# Cargar el dataset modificado
df_modificado = pd.read_csv('../data_en/spam_ham_dataset_modificado.csv')

# Dividir el dataset en conjunto de entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(df_modificado['text'], df_modificado['label_num'], test_size=0.2, random_state=42, stratify=df_modificado['label_num'])

# Dividir el conjunto de entrenamiento en entrenamiento y validación (75% entrenamiento, 25% validación)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42, stratify=y_train)

# Vectorización usando TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')

# Aplicar SMOTE solo en el conjunto de entrenamiento
smote = SMOTE(random_state=42)

# Crear el pipeline
pipeline = ImbPipeline([
    ('tfidf', vectorizer),
    ('smote', smote),
    ('nb', MultinomialNB())
])

# Entrenar el modelo
pipeline.fit(X_train, y_train)

# Guardar el modelo y el vectorizador
joblib.dump(pipeline, '../modelos_y_vectorizadores_en/naive_bayes_smote_pipeline.pkl')


['naive_bayes_smote_pipeline.pkl']

In [2]:

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Cargar el modelo previamente guardado
pipeline = joblib.load('../modelos_y_vectorizadores_en/naive_bayes_smote_pipeline.pkl')

# Cargar el conjunto de datos de prueba
X_test = pd.read_csv('../data_en/X_test.csv')
y_test = pd.read_csv('../data_en/y_test.csv')

# Hacer predicciones en el conjunto de prueba
y_test_pred = pipeline.predict(X_test['text'])

# Evaluar el modelo en el conjunto de prueba
print("Reporte de Clasificación en el conjunto de prueba:")
print(classification_report(y_test['label_num'], y_test_pred))

print("Matriz de Confusión en el conjunto de prueba:")
print(confusion_matrix(y_test['label_num'], y_test_pred))

print("Exactitud en el conjunto de prueba:")
print(accuracy_score(y_test['label_num'], y_test_pred))


Reporte de Clasificación en el conjunto de prueba:
              precision    recall  f1-score   support

           0       0.99      0.95      0.97       706
           1       0.90      0.99      0.94       293

    accuracy                           0.96       999
   macro avg       0.95      0.97      0.96       999
weighted avg       0.97      0.96      0.96       999

Matriz de Confusión en el conjunto de prueba:
[[674  32]
 [  4 289]]
Exactitud en el conjunto de prueba:
0.963963963963964


Los resultados obtenidos al aplicar SMOTE para balancear el dataset muestran una mejora notable en el rendimiento del modelo. Aquí está un análisis detallado de las métricas:

### Análisis del Reporte de Clasificación

El reporte de clasificación después de aplicar SMOTE es el siguiente:

```
              precision    recall  f1-score   support

           0       0.99      0.95      0.97       706
           1       0.90      0.99      0.94       293

    accuracy                           0.96       999
   macro avg       0.95      0.97      0.96       999
weighted avg       0.97      0.96      0.96       999
```

#### **Interpretación:**

1. **Precisión y Recuerdo**:
   - **`ham` (0)**:
     - **Precisión**: 0.99 (99% de las veces que el modelo predice `ham`, es correcto).
     - **Recuerdo**: 0.95 (95% de los verdaderos `ham` son identificados correctamente).
   - **`spam` (1)**:
     - **Precisión**: 0.90 (90% de las veces que el modelo predice `spam`, es correcto).
     - **Recuerdo**: 0.99 (99% de los verdaderos `spam` son identificados correctamente).

2. **F1-Score**:
   - **`ham` (0)**: 0.97, lo cual indica un buen equilibrio entre precisión y recuerdo.
   - **`spam` (1)**: 0.94, que también es bastante bueno y muestra un buen equilibrio.

3. **Exactitud**:
   - La exactitud general del modelo es 0.96 (96%), lo que indica que el modelo hace predicciones correctas el 96% de las veces.

4. **Promedio Macro y Ponderado**:
   - **Macro Avg**: Promedia las métricas sin considerar el soporte. Muestra un buen rendimiento global, con un F1-Score de 0.96.
   - **Weighted Avg**: Promedia las métricas considerando el soporte. El rendimiento ponderado también es alto, con un F1-Score de 0.96.

### Análisis de la Matriz de Confusión

La matriz de confusión es:

```
[[674  32]
 [  4 289]]
```

- **True Negatives (TN)**: 674 (Mensajes `ham` correctamente clasificados como `ham`).
- **False Positives (FP)**: 32 (Mensajes `ham` clasificados incorrectamente como `spam`).
- **False Negatives (FN)**: 4 (Mensajes `spam` clasificados incorrectamente como `ham`).
- **True Positives (TP)**: 289 (Mensajes `spam` correctamente clasificados como `spam`).

### Conclusión

- **Mejoras Notables**:
  - **Recuerdo para `spam` (1)** ha mejorado significativamente (0.99), lo cual es crucial, especialmente si el costo de perder mensajes `spam` es alto.
  - **Precisión para `spam` (1)** también es alta (0.90), lo que indica que el modelo predice `spam` con buena exactitud.

- **Precisión Alta para `ham` (0)**:
  - La precisión para `ham` sigue siendo alta (0.99), y el modelo mantiene un buen recuerdo (0.95).

- **Balance entre Precisión y Recuerdo**:
  - El F1-Score para ambas clases muestra un buen equilibrio, indicando que el modelo está funcionando bien en general.

### Recomendaciones Adicionales

1. **Experimentar con Otros Algoritmos**:
   - Considera probar otros algoritmos de clasificación, como Random Forest, SVM, o redes neuronales, para comparar el rendimiento.

2. **Optimización Adicional**:
   - Ajusta hiperparámetros del modelo o realiza una búsqueda en cuadrícula para mejorar aún más el rendimiento.

3. **Validación Cruzada**:
   - Realiza una validación cruzada para obtener una estimación más robusta del rendimiento del modelo en datos no vistos.

4. **Revisión de Datos**:
   - Asegúrate de revisar el dataset para identificar cualquier posible sesgo o problemas adicionales que puedan afectar el rendimiento del modelo.

En resumen, la aplicación de SMOTE ha tenido un impacto positivo en el rendimiento del modelo, especialmente en el manejo de la clase minoritaria (`spam`), y el modelo muestra un buen rendimiento general.

Para identificar los casos donde el modelo se ha equivocado en el conjunto de prueba, se pueden seguir estos pasos:

Realizar las predicciones en el conjunto de prueba.
Comparar las predicciones con las etiquetas reales.
Identificar y mostrar los casos donde las predicciones no coinciden con las etiquetas reales.

In [10]:
# Identificar los casos donde el modelo se ha equivocado

# Crear un DataFrame con las predicciones y las etiquetas reales
results_df = pd.DataFrame({'text': X_test['text'], 'actual': y_test['label_num'], 'predicted': y_test_pred})

# Filtrar los casos donde las predicciones no coinciden con las etiquetas reales
incorrect_cases = results_df[results_df['actual'] != results_df['predicted']]

# Mostrar los casos incorrectos
print("Casos donde el modelo se ha equivocado:")
print(incorrect_cases)


Casos donde el modelo se ha equivocado:
                                                  text  actual  predicted
6    registration confirmation from spinner . com\r...       0          1
19   http : / / www . pge - texas . com / www / gtt...       0          1
73   your amazon . com order ( # 104 - 9670681 - 03...       0          1
88   thanks from ken walther\r\ni can not begin to ...       0          1
132  this week ' s woodworking tip . . . follow - u...       0          1
232  lacy ' s eye exam\r\nhi bubba !\r\nlacy got he...       0          1
247  your amazon . com order ( # 104 - 9670681 - 03...       0          1
388  fw : whose needs ? ? ? ? ? ? ? ?\r\n> perfect ...       0          1
418  welcome to woodworkingtips . com !\r\n* please...       0          1
448  ? ? ? ? ? ? ? ? erp !\r\n? ????? ? ?  ? ? ????...       1          0
491  i can receive attachments on my hotmail addres...       0          1
566  invitation to dinner\r\nmusic can be started b...       0          

In [11]:
# Mapear las etiquetas numéricas a etiquetas textuales
label_mapping = {0: 'ham', 1: 'spam'}
y_test_mapped = y_test['label_num'].map(label_mapping)
y_test_pred_mapped = pd.Series(y_test_pred).map(label_mapping)

# Identificar los casos donde el modelo se ha equivocado
# Crear un DataFrame con las predicciones y las etiquetas reales
results_df = pd.DataFrame({'text': X_test['text'], 'actual': y_test_mapped, 'predicted': y_test_pred_mapped})

# Filtrar los casos donde las predicciones no coinciden con las etiquetas reales
incorrect_cases = results_df[results_df['actual'] != results_df['predicted']]

# Mostrar los casos incorrectos
print("Casos donde el modelo se ha equivocado:")
print(incorrect_cases)

Casos donde el modelo se ha equivocado:
                                                  text actual predicted
6    registration confirmation from spinner . com\r...    ham      spam
19   http : / / www . pge - texas . com / www / gtt...    ham      spam
73   your amazon . com order ( # 104 - 9670681 - 03...    ham      spam
88   thanks from ken walther\r\ni can not begin to ...    ham      spam
132  this week ' s woodworking tip . . . follow - u...    ham      spam
232  lacy ' s eye exam\r\nhi bubba !\r\nlacy got he...    ham      spam
247  your amazon . com order ( # 104 - 9670681 - 03...    ham      spam
388  fw : whose needs ? ? ? ? ? ? ? ?\r\n> perfect ...    ham      spam
418  welcome to woodworkingtips . com !\r\n* please...    ham      spam
448  ? ? ? ? ? ? ? ? erp !\r\n? ????? ? ?  ? ? ????...   spam       ham
491  i can receive attachments on my hotmail addres...    ham      spam
566  invitation to dinner\r\nmusic can be started b...    ham      spam
585  jeff ' s corner - -

Analizando los casos donde el modelo se ha equivocado, podemos identificar áreas de mejora para ajustar y mejorar el modelo de clasificación de spam. Aquí hay algunas posibles acciones:

1. Analizar los Errores del Modelo
Ham Clasificado como Spam:

Mensajes de confirmación, pedidos de Amazon, correos electrónicos personales, etc., han sido clasificados incorrectamente como spam.
Estos errores pueden estar relacionados con características específicas en el texto que el modelo ha asociado incorrectamente con el spam.
Spam Clasificado como Ham:

Algunos mensajes spam, como aquellos con contenido publicitario explícito, no han sido identificados correctamente.
Esto podría sugerir que el modelo necesita más ejemplos o características específicas para identificar mejor estos patrones.
2. Mejorar la Representación del Texto
N-grams:
Utilizar n-grams (bigramas, trigramas, etc.) en lugar de solo unigrams puede ayudar a capturar más contexto en el texto.
python
Copy code
vectorizer = TfidfVectorizer(ngram_range=(1, 2))  # bigramas
Stemming y Lemmatization:
Aplicar stemming o lemmatization para reducir las palabras a su forma raíz y reducir la dimensionalidad del texto.
3. Ajustar el Modelo
Modelos Más Avanzados:
Probar modelos más avanzados como Random Forest, Gradient Boosting, o incluso modelos basados en redes neuronales.
Considerar el uso de modelos específicos para NLP como BERT o GPT-3.
4. Revisión y Expansión del Dataset
Balanceo del Dataset:
Aunque SMOTE ayuda a balancear el dataset, es posible que se necesite más diversidad en los ejemplos de entrenamiento.
Revisar y Etiquetar Datos Manualmente:
Revisar manualmente algunos de los errores para mejorar la calidad del dataset.
5. Tuning de Hiperparámetros
Grid Search o Random Search:
Realizar una búsqueda de hiperparámetros para encontrar los mejores parámetros para el modelo actual.
6. Filtrado de Características
Eliminar Caracteres Especiales:
Eliminar caracteres especiales que puedan no aportar valor significativo.
python
Copy code
df_modificado['text'] = df_modificado['text'].str.replace('[^a-zA-Z0-9\s]', '')


## Examinemos algunos nuevos casos

In [3]:
# Ejemplo de texto nuevo
new_example = ["this is your last opportunity"]

# Usar el pipeline para predecir la clase del nuevo ejemplo
prediction = pipeline.predict(new_example)

# Convertir la predicción a etiqueta legible
label_map = {0: 'ham', 1: 'spam'}
predicted_label = label_map[prediction[0]]

print(f"El mensaje '{new_example[0]}' ha sido clasificado como: {predicted_label}")


El mensaje 'this is your last opportunity' ha sido clasificado como: spam


In [4]:
# Ejemplo de texto nuevo
new_example = ["Call me if you are alone"]

# Usar el pipeline para predecir la clase del nuevo ejemplo
prediction = pipeline.predict(new_example)

# Convertir la predicción a etiqueta legible
label_map = {0: 'ham', 1: 'spam'}
predicted_label = label_map[prediction[0]]

print(f"El mensaje '{new_example[0]}' ha sido clasificado como: {predicted_label}")

El mensaje 'Call me if you are alone' ha sido clasificado como: ham


In [5]:
# Ejemplo de texto nuevo
new_example = ["Bring some beers"]

# Usar el pipeline para predecir la clase del nuevo ejemplo
prediction = pipeline.predict(new_example)

# Convertir la predicción a etiqueta legible
label_map = {0: 'ham', 1: 'spam'}
predicted_label = label_map[prediction[0]]

print(f"El mensaje '{new_example[0]}' ha sido clasificado como: {predicted_label}")

El mensaje 'Bring some beers' ha sido clasificado como: spam


In [6]:
# Ejemplo de texto nuevo
new_example = ["I forgot to tell you something"]

# Usar el pipeline para predecir la clase del nuevo ejemplo
prediction = pipeline.predict(new_example)

# Convertir la predicción a etiqueta legible
label_map = {0: 'ham', 1: 'spam'}
predicted_label = label_map[prediction[0]]

print(f"El mensaje '{new_example[0]}' ha sido clasificado como: {predicted_label}")

El mensaje 'I forgot to tell you something' ha sido clasificado como: ham
